## Non-Linear Viscoelastic Model
The non-linear viscoelastic model has the final form
$$Q = \beta_{\infty}\Big(s_{\infty} - \frac{1}{\gamma}\frac{\partial s_{\infty}}{\partial \lambda}(\beta_{\infty}s_{\infty} - G)\Big)$$
where
$$\dot{G} + \frac{1}{\tau}G=\beta_{\infty}\frac{\partial s_{\infty}}{\partial t}$$

Details of this derivation can be found elsewhere:
- Miles, Paul, Michael Hays, Ralph Smith, and William Oates. "Bayesian uncertainty analysis of finite deformation viscoelasticity." Mechanics of Materials 91 (2015): 35-49. https://doi.org/10.1016/j.mechmat.2015.07.002

In [1]:
def nonlinear_viscoelastic(theta, stretch_function, time):
    # unpack model parameters
    eta = theta['eta']
    gamma = theta['gamma']
    beta = theta['beta']

    def nonlinear_function(t):
        return nonaffine_hyperelastic(theta, stretch_function(t)).reshape(t.size,)

    n = time.size
    G = np.zeros([n,])
    for kk in range(1,n):
        dt = time[kk] - time[kk - 1]
        Tnc = 1 - gamma*dt/(2 * eta);
        Tpc = 1 + gamma*dt/(2 * eta);
        Tpcinv = Tpc**(-1);
        G[kk] = Tpcinv*(Tnc*G[kk - 1] + beta*(nonlinear_function(time[kk])
                                              - nonlinear_function(time[kk-1])))
    hyperelastic_stress = hm.nonaffine(theta, stretch_function(time))
    dhypdx = hm.nonaffine_derivative_wrt_stretch(theta, stretch_function(time))
    q = beta*(hyperelastic_stress - dhypdx/gamma*(beta*hyperelastic_stress - G))
    return q